<div style="nosxt-align:cennosr; font-size:26px; color:green;">Renta Fija. Matriz de decisión</div>

Si no compartimos el conocimiento no sirve para nada. [Michio Kaku](https://www.younuestrobe.com/watch?v=6rDxlolYUQw)



**Descarga de cotizaciones.**  
Para obtener datos históricos de precios, puedes:

Descargarlos directamente desde las gestoras de los fondos.
Utilizar plataformas públicas como Yahoo Finance mediante la librería yfinance.
yfinance es una herramienta popular en análisis financiero que facilita el acceso a datos históricos considerados de dominio público. Sin embargo, estos datos pueden presentar pequeñas diferencias con respecto a las fuentes oficiales. Por ello, para decisiones críticas, siempre se recomienda contrastar la información con la documentación oficial de la gestora.

⚠️ **Precauciones.**
Este proyecto utiliza yfinance (con licencia bajo la [Apache License 2.0](https://www.apache.org/licenses/LICENSE-2.0).
), una interfaz para descargar datos financieros desde Yahoo Finance. Ten en cuenta lo siguiente:

Los datos descargados son exclusivamente para uso personal.
No está permitido redistribuirlos o venderlos.
Su uso en contextos educativos o sin ánimo de lucro suele ser aceptable, siempre que se cumplan estas condiciones.
Evita realizar peticiones excesivas o abusivas que puedan violar los términos de uso del servicio.

# Ejecutar si estamos en Google Colaboratory.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install pandas numpy openpyxl yfinance


Tus rutas tienen que empezar por '/content/drive/MyDrive/
'.

In [ ]:
# Versión 4 Optimizada

import pandas as pd
import yfinance as yf
import numpy as np
from datetime import datetime, timedelta
from openpyxl import load_workbook
from dataclasses import dataclass
from typing import Optional, Literal, Dict, Tuple, List

RRegime = Literal["alcista", "bajista", "correccion", "recesion", "lateral", "unknown"]


# =============================================================================
# UTILIDADES COMUNES
# =============================================================================

def excel_date_to_datetime(value):
    """Convierte cualquier formato de fecha en datetime."""
    if value is None or pd.isna(value):
        return None

    if isinstance(value, (pd.Timestamp, datetime)):
        return value if isinstance(value, datetime) else value.to_pydatetime()

    if isinstance(value, (int, float)):
        try:
            return datetime(1899, 12, 30) + timedelta(days=float(value))
        except:
            return None

    if isinstance(value, str):
        try:
            return pd.to_datetime(value).to_pydatetime()
        except:
            return None

    return None


def normalize_columns(df: pd.DataFrame) -> pd.DataFrame:
    """Normaliza nombres de columnas (minúsculas, sin espacios)."""
    df.columns = df.columns.astype(str).str.strip().str.lower()
    return df


# =============================================================================
# GESTOR DE PRECIOS (Elimina duplicación)
# =============================================================================

class PriceManager:
    """Centraliza toda la lógica de descarga de precios."""

    @staticmethod
    def get_price(ticker: str, date: Optional[datetime] = None) -> Optional[float]:
        """Obtiene precio en una fecha específica o actual."""
        try:
            if date is None:
                data = yf.download(ticker, period="5d", auto_adjust=False, progress=False)
            else:
                data = yf.download(
                    ticker,
                    start=date,
                    end=date + timedelta(days=3),
                    auto_adjust=False,
                    progress=False
                )

            if not data.empty:
                return float(data["Close"].iloc[-1 if date is None else 0].item())
        except:
            return None
        return None

    @staticmethod
    def download_history(ticker: str, period: str = "5y") -> Optional[pd.Series]:
        """Descarga histórico de precios y retorna serie de retornos."""
        try:
            data = yf.download(ticker, period=period, auto_adjust=True, progress=False)

            if isinstance(data.columns, pd.MultiIndex):
                data = data["Close"][ticker]
            else:
                data = data["Close"]

            returns = data.pct_change().dropna()

            if isinstance(returns, pd.DataFrame):
                returns = returns.iloc[:, 0]

            return returns
        except:
            return None


# =============================================================================
# CALCULADOR DE MÉTRICAS (Elimina duplicación)
# =============================================================================

class MetricsCalculator:
    """Calcula todas las métricas financieras de forma centralizada."""

    @staticmethod
    def cagr(returns: pd.Series) -> float:
        """Tasa de crecimiento anual compuesta."""
        if returns is None or returns.empty or len(returns) < 2:
            return np.nan
        total_return = (1 + returns).prod()
        years = len(returns) / 252
        return total_return ** (1 / years) - 1 if years > 0 else np.nan

    @staticmethod
    def volatility(returns: pd.Series) -> float:
        """Volatilidad anualizada."""
        return returns.std() * np.sqrt(252) if returns is not None else np.nan

    @staticmethod
    def sharpe(returns: pd.Series, rf: float = 0.01) -> float:
        """Ratio de Sharpe."""
        vol = MetricsCalculator.volatility(returns)
        if vol is None or vol == 0:
            return np.nan
        excess = returns.mean() * 252 - rf
        return excess / vol

    @staticmethod
    def sortino(returns: pd.Series, rf: float = 0.01) -> float:
        """Ratio de Sortino."""
        downside = returns[returns < 0].std() * np.sqrt(252)
        if downside is None or downside == 0:
            return np.nan
        excess = returns.mean() * 252 - rf
        return excess / downside

    @staticmethod
    def max_drawdown(returns: pd.Series) -> float:
        """Máxima caída desde máximos."""
        if returns is None or returns.empty:
            return np.nan
        cumulative = (1 + returns).cumprod()
        peak = cumulative.cummax()
        dd = (cumulative - peak) / peak
        return dd.min()

    @staticmethod
    def beta_alpha_r2(fund_returns: pd.Series, bench_returns: pd.Series) -> Tuple[float, float, float]:
        """Calcula Beta, Alpha y R² respecto a un benchmark."""
        if fund_returns is None or bench_returns is None:
            return np.nan, np.nan, np.nan

        df = pd.concat([fund_returns, bench_returns], axis=1).dropna()
        df.columns = ["fund", "bench"]

        if len(df) < 30:
            return np.nan, np.nan, np.nan

        cov = df.cov().iloc[0, 1]
        var = df["bench"].var()

        beta = cov / var if var != 0 else np.nan
        alpha = (df["fund"].mean() * 252) - beta * (df["bench"].mean() * 252)
        r2 = df.corr().iloc[0, 1] ** 2

        return beta, alpha, r2

    @staticmethod
    def tracking_error(fund_returns: pd.Series, bench_returns: pd.Series) -> float:
        """Error de seguimiento."""
        diff = fund_returns - bench_returns
        return diff.std() * np.sqrt(252)

    @staticmethod
    def information_ratio(fund_returns: pd.Series, bench_returns: pd.Series) -> float:
        """Ratio de información."""
        te = MetricsCalculator.tracking_error(fund_returns, bench_returns)
        if te is None or te == 0:
            return np.nan
        excess = (fund_returns.mean() - bench_returns.mean()) * 252
        return excess / te


# =============================================================================
# CLASIFICADORES (Buckets y Cuadrantes)
# =============================================================================

class AssetClassifier:
    """Clasifica activos en buckets de duración, calidad y cuadrantes."""

    @staticmethod
    def duration_bucket(duration: float) -> str:
        """Clasifica por duración."""
        if pd.isna(duration):
            return "unknown"
        try:
            d = float(duration)
        except:
            return "unknown"

        if d <= 1:
            return "ultra_corta"
        elif d <= 3:
            return "corta"
        elif d <= 5:
            return "media"
        elif d <= 10:
            return "larga"
        else:
            return "ultra_larga"

    @staticmethod
    def quality_bucket(rating: str) -> str:
        """Clasifica por calidad crediticia."""
        if rating is None or pd.isna(rating):
            return "unknown"

        r = str(rating).upper().replace(" ", "")

        if r.startswith(("AAA", "AA", "A")):
            return "alta"
        elif r.startswith(("BBB", "BAA")):
            return "media"
        else:
            return "baja"

    @staticmethod
    def assign_quadrant(duration_bucket: str, quality_bucket: str) -> int:
        """Asigna cuadrante basado en duración y calidad."""
        dur_map = {
            "ultra_corta": "short", "corta": "short",
            "larga": "long", "ultra_larga": "long",
            "media": "medium"
        }

        qual_map = {
            "alta": "high",
            "media": "low", "baja": "low"
        }

        d = dur_map.get(duration_bucket, "unknown")
        q = qual_map.get(quality_bucket, "unknown")

        if d == "short" and q == "high":
            return 1
        if d == "long" and q == "high":
            return 2
        if d == "short" and q == "low":
            return 3
        if d == "long" and q == "low":
            return 4
        return 0


# =============================================================================
# DIAGNÓSTICOS (Centralizado)
# =============================================================================

class DiagnosticEngine:
    """Motor de diagnósticos cualitativos."""

    THRESHOLDS = {
        "sharpe": [(1, "Excelente"), (0.5, "Bueno"), (0, "Débil")],
        "volatilidad": [(0.02, "Muy baja"), (0.05, "Moderada"), (float('inf'), "Elevada")],
        "max_drawdown": [(-0.05, "Controladas"), (-0.15, "Moderadas"), (-float('inf'), "Profundas")],
        "beta": [(0.3, "Baja sensibilidad"), (0.7, "Moderada"), (float('inf'), "Alta")],
        "r2": [(0.2, "Independiente"), (0.5, "Moderada"), (float('inf'), "Alta dependencia")],
        "cagr": [(0.05, "Sólido"), (0.02, "Moderado"), (0, "Débil")],
        "tracking_error": [(0.03, "Muy estable"), (0.07, "Moderada"), (float('inf'), "Alta")],
        "information_ratio": [(0.5, "Generación de alfa"), (0, "Ligera generación"), (-float('inf'), "No genera")]
    }

    @classmethod
    def diagnose(cls, metric: str, value: float) -> str:
        """Genera diagnóstico cualitativo para una métrica."""
        if value is None or pd.isna(value):
            return "Sin datos suficientes."

        thresholds = cls.THRESHOLDS.get(metric, [])

        for threshold, label in thresholds:
            if (metric in ["max_drawdown"] and value > threshold) or \
               (metric not in ["max_drawdown"] and value < threshold):
                return label

        return "Sin diagnóstico disponible."


# =============================================================================
# CONTEXTO DE MERCADO
# =============================================================================

@dataclass
class MarketContext:
    regime: Regime
    as_of_date: Optional[datetime] = None
    comment: Optional[str] = None


# =============================================================================
# SISTEMA PRINCIPAL OPTIMIZADO
# =============================================================================

class FixedIncomePortfolioSystem:
    """Sistema integrado de análisis de cartera de renta fija."""

    def __init__(self, excel_path: str, benchmark: str = "^GSPC"):
        self.excel_path = excel_path
        self.benchmark = benchmark
        self.df_fondos = None
        self.df_movs = None
        self.df = None
        self.quality_df = None

    # -------------------------------------------------------------------------
    # CARGA Y ENRIQUECIMIENTO
    # -------------------------------------------------------------------------

    def load_data(self):
        """Carga datos del Excel."""
        self.df_fondos = pd.read_excel(self.excel_path, sheet_name="Fondos")
        self.df_movs = pd.read_excel(self.excel_path, sheet_name="Movimientos")

    def enrich_fondos(self):
        """Enriquece la hoja Fondos con precios y métricas básicas."""
        df = self.df_fondos.copy()
        df["date"] = pd.to_datetime(df["date"], dayfirst=True, errors="coerce")

        if df["date"].isna().any():
            raise ValueError("Hay fechas inválidas en la hoja Fondos.")

        # Aplicar cálculos de precios de forma vectorizada donde sea posible
        results = []
        for _, row in df.iterrows():
            ticker = row["Ticker"]
            fecha_inv = row["date"]
            amount = row["amount"]

            price_buy = PriceManager.get_price(ticker, fecha_inv)
            price_now = PriceManager.get_price(ticker)

            if price_buy and price_now:
                invested = amount * price_buy
                current = amount * price_now
                ret = (price_now / price_buy) - 1
                años = max((pd.Timestamp("today") - fecha_inv).days / 365.25, 1/365)
                cagr = (price_now / price_buy) ** (1 / años) - 1
            else:
                invested = current = ret = años = cagr = None

            results.append({
                "nav": price_buy,
                "nav_current": price_now,
                "Valor_invertido": invested,
                "Valor_actual": current,
                "Rentabilidad": ret,
                "Años": años,
                "CAGR": cagr
            })

        # Asignar resultados
        for key in results[0].keys():
            df[key] = [r[key] for r in results]

        # Calcular pesos
        total_actual = df["Valor_actual"].sum(skipna=True)
        df["Peso"] = df["Valor_actual"] / total_actual if total_actual else 0

        self.df_fondos = df
        print(f"\n✅ Fondos enriquecidos: {df['Ticker'].count()} fondos procesados")
        print(f"Rentabilidad media: {df['Rentabilidad'].mean()*100:.2f}%")

        return df

    def process(self):
        """Procesa datos combinando fondos y movimientos."""
        df = self.df_fondos.merge(self.df_movs, on=["ISIN", "Ticker"], how="left")

        # Clasificar en buckets y cuadrantes
        df["bucket_duracion_calc"] = df["Duracion años"].apply(AssetClassifier.duration_bucket)
        df["bucket_calidad_calc"] = df["Rating_avg"].apply(AssetClassifier.quality_bucket)
        df["quadrant"] = df.apply(
            lambda row: AssetClassifier.assign_quadrant(
                row["bucket_duracion_calc"],
                row["bucket_calidad_calc"]
            ), axis=1
        )

        # Normalizar nombres de columnas duplicadas
        if "Nombre_x" in df.columns:
            df = df.rename(columns={"Nombre_x": "Nombre"})
        if "Nombre_y" in df.columns:
            df = df.drop(columns=["Nombre_y"])

        self.df = df

    # -------------------------------------------------------------------------
    # CLASIFICACIÓN DE TIPO DE MERCADO (nuevo algoritmo)
    # -------------------------------------------------------------------------

    def classify_market_regime(self, window_vol=20, smooth_vol=5, min_days=5):
        """Clasifica tipo de mercado usando el benchmark."""
        returns = PriceManager.download_history(self.benchmark)
        if returns is None or returns.empty:
            print("⚠️ No se pudo descargar histórico del benchmark.")
            self.market_df = None
            return None

        df = pd.DataFrame(index=returns.index)
        df["Close"] = (1 + returns).cumprod()

        df["MA50"] = df["Close"].rolling(50).mean()
        df["MA200"] = df["Close"].rolling(200).mean()

        df["Max_To_Date"] = df["Close"].cummax()
        df["Drawdown"] = (df["Close"] - df["Max_To_Date"]) / df["Max_To_Date"]

        df["Log_Returns"] = np.log(df["Close"] / df["Close"].shift(1))
        df["Volatility"] = df["Log_Returns"].rolling(window_vol).std() * np.sqrt(252)
        df["Vol_Smooth"] = df["Volatility"].rolling(smooth_vol).mean()

        vol60 = df["Vol_Smooth"].quantile(0.60)
        vol75 = df["Vol_Smooth"].quantile(0.75)

        df["Market_Type"] = "lateral"

        df.loc[
            (df["Drawdown"] < -0.20) &
            (df["Vol_Smooth"] > vol75) &
            (df["MA50"] < df["MA200"]),
            "Market_Type"
        ] = "recesion"

        df.loc[
            (df["Drawdown"] <= -0.10) &
            (df["Drawdown"] > -0.20) &
            (df["MA50"] < df["MA200"]),
            "Market_Type"
        ] = "bajista"

        df.loc[
            (df["Drawdown"] <= -0.10) &
            (df["Drawdown"] > -0.20) &
            (df["MA50"] >= df["MA200"]),
            "Market_Type"
        ] = "correccion"

        df.loc[
            (df["MA50"] > df["MA200"]) &
            (df["Drawdown"] > -0.10) &
            (df["Vol_Smooth"] < vol60),
            "Market_Type"
        ] = "alcista"

        df["Market_Type"] = self._smooth_market_type(df["Market_Type"], min_days=min_days)

        self.market_df = df
        print("✅ Clasificación de mercado completada.")
        return df


    def _smooth_market_type(self, serie, min_days=5):
        """Suaviza serie categórica exigiendo persistencia mínima."""
        serie = serie.copy()
        salida = serie.copy()
        actual = serie.iloc[0]
        contador = 1

        for i in range(1, len(serie)):
            if serie.iloc[i] == actual:
                contador += 1
            else:
                if contador < min_days:
                    salida.iloc[i-contador:i] = actual
                actual = serie.iloc[i]
                contador = 1

        if contador < min_days:
            salida.iloc[len(serie)-contador:] = actual

        return salida


    def get_market_type_for_date(self, date: datetime) -> str:
        """Devuelve el tipo de mercado para una fecha concreta."""
        if self.market_df is None:
            return "unknown"

        date = pd.to_datetime(date)
        if date not in self.market_df.index:
            return "unknown"

        return self.market_df.loc[date, "Market_Type"]


    # -------------------------------------------------------------------------
    # ANÁLISIS DE CALIDAD
    # -------------------------------------------------------------------------

    def analyze_quality(self):
        """Analiza métricas de calidad para todos los fondos."""
        benchmark_returns = PriceManager.download_history(self.benchmark)

        results = []
        for _, row in self.df_fondos.iterrows():
            ticker = row["Ticker"]
            fund_returns = PriceManager.download_history(ticker)

            if fund_returns is None:
                metrics = {key: np.nan for key in [
                    "Volatilidad", "Sharpe", "Sortino", "CAGR", "Max_Drawdown",
                    "Beta", "Alpha", "R2", "Tracking_Error", "Information_Ratio"
                ]}
            else:
                beta, alpha, r2 = MetricsCalculator.beta_alpha_r2(fund_returns, benchmark_returns)

                metrics = {
                    "Volatilidad": MetricsCalculator.volatility(fund_returns),
                    "Sharpe": MetricsCalculator.sharpe(fund_returns),
                    "Sortino": MetricsCalculator.sortino(fund_returns),
                    "CAGR": MetricsCalculator.cagr(fund_returns),
                    "Max_Drawdown": MetricsCalculator.max_drawdown(fund_returns),
                    "Beta": beta,
                    "Alpha": alpha,
                    "R2": r2,
                    "Tracking_Error": MetricsCalculator.tracking_error(fund_returns, benchmark_returns),
                    "Information_Ratio": MetricsCalculator.information_ratio(fund_returns, benchmark_returns)
                }

            results.append({"Ticker": ticker, **metrics})

        self.quality_df = pd.DataFrame(results)
        return self.quality_df

    def analyze_portfolio_global(self):
        """Analiza la cartera global construyendo la serie temporal."""
        print("\n" + "="*70)
        print(" INFORME GLOBAL DE LA CARTERA")
        print("="*70 + "\n")

        # Construir serie de cartera ponderada
        series_list = []
        weight_list = []

        for _, row in self.df_fondos.iterrows():
            ticker = row["Ticker"]
            weight = row.get("Peso", 0)

            if weight == 0:
                continue

            returns = PriceManager.download_history(ticker)
            if returns is not None and not returns.empty:
                series_list.append(returns)
                weight_list.append(weight)

        if not series_list:
            print("⚠️ No se pudo construir la serie de cartera.")
            return

        # Alinear series
        df = pd.concat(series_list, axis=1).dropna()

        # Serie ponderada de retornos
        portfolio_returns = (df * weight_list).sum(axis=1)
        portfolio_nav = (1 + portfolio_returns).cumprod()

        # Calcular métricas
        valor_inicial = portfolio_nav.iloc[0]
        valor_final = portfolio_nav.iloc[-1]
        ret_total = (valor_final / valor_inicial) - 1
        cagr = MetricsCalculator.cagr(portfolio_returns)
        vol = MetricsCalculator.volatility(portfolio_returns)
        mdd = MetricsCalculator.max_drawdown(portfolio_returns)
        sharpe = MetricsCalculator.sharpe(portfolio_returns)
        sortino = MetricsCalculator.sortino(portfolio_returns)

        # Mostrar resultados
        print(f"Valor inicial: {valor_inicial:.4f}")
        print(f"Valor final:   {valor_final:.4f}")
        print(f"Rentabilidad total: {ret_total*100:.2f}%")
        print(f"CAGR: {cagr*100:.2f}%")
        print(f"Volatilidad: {vol*100:.2f}%")
        print(f"Max Drawdown: {mdd*100:.2f}%")
        print(f"Sharpe: {sharpe:.3f}")
        print(f"Sortino: {sortino:.3f}")

        # Diagnóstico cualitativo
        print("\nDiagnóstico cualitativo:")

        if ret_total > 0.05:
            print("- Rentabilidad total sólida.")
        elif ret_total > 0:
            print("- Rentabilidad positiva pero modesta.")
        else:
            print("- Rentabilidad negativa, revisar composición.")

        if cagr > 0.04:
            print("- Crecimiento anual compuesto saludable.")
        elif cagr > 0.01:
            print("- Crecimiento moderado.")
        else:
            print("- Crecimiento débil o nulo.")

        if vol < 0.03:
            print("- Volatilidad muy baja, perfil conservador.")
        elif vol < 0.07:
            print("- Volatilidad moderada.")
        else:
            print("- Volatilidad elevada, revisar riesgo.")

        if mdd > -0.05:
            print("- Caídas muy controladas.")
        elif mdd > -0.15:
            print("- Caídas moderadas.")
        else:
            print("- Caídas profundas, riesgo elevado.")

        if sharpe > 1:
            print("- Excelente relación rentabilidad/riesgo.")
        elif sharpe > 0.5:
            print("- Buena relación rentabilidad/riesgo.")
        else:
            print("- Rentabilidad ajustada al riesgo débil.")

        if sortino > 1:
            print("- Buen control de caídas.")
        else:
            print("- Control de caídas mejorable.")

    # -------------------------------------------------------------------------
    # RESÚMENES Y DIAGNÓSTICOS
    # -------------------------------------------------------------------------

    def summary_by_quadrant(self) -> pd.DataFrame:
        """Resumen agregado por cuadrantes."""
        if self.df is None:
            raise ValueError("Ejecuta process() antes del resumen.")

        quad = self.df.groupby("quadrant").agg(
            total_value=("Valor_actual", "sum"),
            total_invested=("Valor_invertido", "sum"),
            weight=("Peso", "sum"),
            avg_return=("Rentabilidad", "mean")
        )

        quad["accumulated_return"] = (
            (quad["total_value"] - quad["total_invested"]) / quad["total_invested"]
        )

        return quad

    def diagnose(self, market: MarketContext) -> Tuple[pd.DataFrame, List[str]]:
        """Genera diagnóstico basado en el contexto de mercado."""
        quad = self.summary_by_quadrant()

        messages = [
            f"Diagnóstico a fecha: {market.as_of_date.date() if market.as_of_date else 'N/A'}"
        ]

        if market.comment:
            messages.append(f"Comentario: {market.comment}")

        w = {q: quad.loc[q, "weight"] if q in quad.index else 0 for q in [0,1,2,3,4]}

        regime_messages = {
            "rates_up": ("Entorno: subidas de tipos → favorece Cuadrante 1.",
                        4, "Tienes exposición al Cuadrante 4: muy agresivo en este entorno."),
            "rates_down": ("Entorno: bajadas de tipos → Cuadrante 2 puede aportar plusvalías.", None, None),
            "stable_strong": ("Entorno estable → Cuadrante 3 puede ser atractivo por cupones.", None, None),
            "recovery": ("Recuperación → algo de Cuadrante 4 puede ser táctico.", None, None)
        }

        if market.regime in regime_messages:
            msg, warn_q, warn_msg = regime_messages[market.regime]
            messages.append(msg)
            if warn_q and w[warn_q] > 0:
                messages.append(warn_msg)
        else:
            messages.append("Régimen desconocido.")

        return quad, messages

    # -------------------------------------------------------------------------
    # ESCRITURA EN EXCEL
    # -------------------------------------------------------------------------

    def write_to_excel(self):
        """Escribe resultados enriquecidos en Excel."""
        wb = load_workbook(self.excel_path)

        # Limpiar columnas extra
        for sheet_name in ["Fondos", "Indicadores"]:
            if sheet_name in wb.sheetnames:
                ws = wb[sheet_name]
                self._clean_extra_columns(ws, max_keep=50)
            elif sheet_name == "Indicadores":
                wb.create_sheet("Indicadores")

        # Escribir Fondos
        print("\n📝 Actualizando hoja 'Fondos'...")
        self._write_fondos_sheet(wb["Fondos"])

        # Escribir Indicadores
        if self.quality_df is not None:
            print("📝 Actualizando hoja 'Indicadores'...")
            self._write_indicators_sheet(wb["Indicadores"])

        wb.save(self.excel_path)
        print(f"✅ Excel actualizado correctamente: {self.excel_path}")
        print(f"   • Hoja 'Fondos': {len(self.df_fondos)} registros actualizados")
        if self.quality_df is not None:
            print(f"   • Hoja 'Indicadores': {len(self.quality_df)} registros actualizados")

    def _clean_extra_columns(self, ws, max_keep: int = 50):
        """Elimina columnas sobrantes."""
        if ws.max_column > max_keep:
            ws.delete_cols(max_keep + 1, ws.max_column - max_keep)

    def _write_fondos_sheet(self, ws):
        """Escribe datos en hoja Fondos."""
        cols = ["nav", "nav_current", "Valor_invertido", "Valor_actual",
                "Rentabilidad", "CAGR", "Peso"]
        self._write_sheet_data(ws, self.df_fondos, cols)

    def _write_indicators_sheet(self, ws):
        """Escribe datos en hoja Indicadores."""
        cols = ["ISIN", "Nombre", "Clase", "date", "Volatilidad", "Sharpe",
                "Sortino", "Max_Drawdown", "Beta", "Alpha", "R2",
                "Tracking_Error", "Information_Ratio"]

        # Combinar metadatos con indicadores
        df_info = self.df_fondos.set_index("Ticker")
        df_combined = self.quality_df.copy()

        for _, row in df_combined.iterrows():
            ticker = row["Ticker"]
            if ticker in df_info.index:
                info = df_info.loc[ticker]
                if isinstance(info, pd.DataFrame):
                    info = info.iloc[0]

        self._write_sheet_data(ws, df_combined, cols)

    def _write_sheet_data(self, ws, df, cols):
        """Método auxiliar para escribir datos en una hoja."""
        col_map = {}
        for c in range(1, 51):
            header = ws.cell(row=1, column=c).value
            if header:
                col_map[header] = c

        next_free_col = 1
        while next_free_col <= 50 and ws.cell(row=1, column=next_free_col).value:
            next_free_col += 1

        for col in cols:
            if col not in col_map and col in df.columns:
                if next_free_col > 50:
                    raise ValueError(f"No hay columnas libres para '{col}'")
                ws.cell(row=1, column=next_free_col, value=col)
                col_map[col] = next_free_col
                next_free_col += 1

        for row_idx, (_, row) in enumerate(df.iterrows(), start=2):
            for col in cols:
                if col in col_map and col in df.columns:
                    ws.cell(row=row_idx, column=col_map[col], value=row.get(col, None))

    # -------------------------------------------------------------------------
    # INFORMES
    # -------------------------------------------------------------------------

    def generate_reports(self, market: MarketContext):
        """Genera informes completos de la cartera."""
        quad, messages = self.diagnose(market)

        print("\n" + "="*70)
        print(" INFORME DE CARTERA DE RENTA FIJA")
        print("="*70 + "\n")

        print("Distribución por cuadrantes:")
        print(quad.to_string())
        print("\n")

        print("Diagnóstico:")
        for m in messages:
            print(f"- {m}")
        print("\n")

        # Informes detallados
        if self.quality_df is not None:
            self._print_matriz_seleccion_completa()
            self._print_diagnostico_matriz()
            self._print_informe_por_fondo()
            self._print_matriz_decision()
            self._print_diagnostico_cuadrante()
            self._print_recomendacion_global()

    def _print_matriz_seleccion_completa(self):
        """Imprime las dos tablas de la matriz de selección."""
        df_sel = normalize_columns(self.quality_df.copy())
        df_meta = normalize_columns(self.df_fondos[["Ticker", "ISIN", "Nombre", "Clase"]].copy())

        df_combined = df_sel.merge(df_meta.drop_duplicates(), on="ticker", how="left")
        df_combined = df_combined.sort_values(by="sharpe", ascending=False)

        # TABLA A
        print("\n" + "="*70)
        print(" MATRIZ DE SELECCIÓN — TABLA A (Métricas principales)")
        print("="*70 + "\n")

        cols_A = ["ticker", "isin", "nombre", "clase", "volatilidad", "sharpe", "sortino", "cagr"]
        cols_A = [c for c in cols_A if c in df_combined.columns]
        print(df_combined[cols_A].to_string(index=False))

        # TABLA B
        print("\n" + "="*70)
        print(" MATRIZ DE SELECCIÓN — TABLA B (Riesgo de mercado)")
        print("="*70 + "\n")

        cols_B = ["ticker", "max_drawdown", "beta", "alpha", "r2", "tracking_error", "information_ratio"]
        cols_B = [c for c in cols_B if c in df_combined.columns]
        print(df_combined[cols_B].to_string(index=False))

    def _print_diagnostico_matriz(self):
        """Imprime diagnóstico detallado de cada fondo en la matriz."""
        df_sel = normalize_columns(self.quality_df.copy())
        df_meta = normalize_columns(self.df_fondos[["Ticker", "ISIN", "Nombre"]].copy())

        df_combined = df_sel.merge(df_meta.drop_duplicates(), on="ticker", how="left")
        df_combined = df_combined.sort_values(by="sharpe", ascending=False)

        print("\n" + "="*70)
        print(" DIAGNÓSTICO DE MATRIZ DE SELECCIÓN")
        print("="*70 + "\n")

        for _, row in df_combined.iterrows():
            nombre = row.get('nombre', 'N/D')
            ticker = row.get('ticker')
            isin = row.get('isin', 'N/D')

            print(f"Fondo: {nombre}  ({ticker})")
            print(f"  ISIN: {isin}")
            print("  --- Diagnóstico ---")

            # Diagnósticos
            sharpe = row.get("sharpe")
            if sharpe is not None and not pd.isna(sharpe):
                if sharpe > 1:
                    print("    • Excelente Sharpe (muy buena relación rentabilidad/riesgo).")
                elif sharpe > 0.5:
                    print("    • Sharpe aceptable.")
                else:
                    print("    • Sharpe débil.")

            vol = row.get("volatilidad")
            if vol is not None and not pd.isna(vol):
                if vol < 0.02:
                    print("    • Volatilidad muy baja (perfil conservador).")
                elif vol < 0.05:
                    print("    • Volatilidad moderada.")
                else:
                    print("    • Volatilidad elevada.")

            mdd = row.get("max_drawdown")
            if mdd is not None and not pd.isna(mdd):
                if mdd > -0.05:
                    print("    • Caídas muy controladas.")
                elif mdd > -0.15:
                    print("    • Caídas moderadas.")
                else:
                    print("    • Caídas profundas (riesgo elevado).")

            ir = row.get("information_ratio")
            if ir is not None and not pd.isna(ir):
                if ir > 0.5:
                    print("    • Generación de alfa consistente.")
                elif ir > 0:
                    print("    • Ligera generación de alfa.")
                else:
                    print("    • No genera alfa respecto al benchmark.")

            print("-" * 60)

    def _print_informe_por_fondo(self):
        """Imprime informe detallado de cada fondo con todas las métricas."""
        df_sel = normalize_columns(self.quality_df.copy())
        df_meta = normalize_columns(self.df_fondos[["Ticker", "ISIN", "Nombre", "Clase"]].copy())

        df_combined = df_sel.merge(df_meta.drop_duplicates(), on="ticker", how="left")
        df_combined = df_combined.sort_values(by="sharpe", ascending=False)

        print("\n" + "="*70)
        print(" INFORME POR FONDO")
        print("="*70 + "\n")

        for _, row in df_combined.iterrows():
            print(f"Fondo: {row.get('nombre', 'N/D')}")
            print(f"  ISIN:   {row.get('isin', 'N/D')}")
            print(f"  Clase:  {row.get('clase', 'N/D')}")
            print(f"  Ticker: {row.get('ticker', 'N/D')}")
            print("  --- Métricas ---")

            for metric in ["volatilidad", "sharpe", "sortino", "cagr",
                          "max_drawdown", "beta", "alpha", "r2",
                          "tracking_error", "information_ratio"]:
                val = row.get(metric, None)
                if isinstance(val, (int, float)) and not pd.isna(val):
                    print(f"    {metric}: {val:.4f}")
                else:
                    print(f"    {metric}: N/D")

            print("-" * 60)

    def _print_matriz_decision(self):
        """Imprime matriz de decisión con cuadrantes."""
        df_sel = normalize_columns(self.quality_df.copy())
        df_meta = normalize_columns(self.df_fondos[["Ticker", "Nombre", "Clase"]].copy())
        df_quad = normalize_columns(self.df[["Ticker", "quadrant"]].copy())

        df_combined = df_sel.merge(df_meta, on="ticker", how="left")
        df_combined = df_combined.merge(df_quad, on="ticker", how="left")
        df_combined = df_combined.sort_values(by="sharpe", ascending=False)

        print("\n" + "="*70)
        print(" MATRIZ DE DECISIÓN (CUADRANTES)")
        print("="*70 + "\n")

        cols = ["ticker", "nombre", "clase", "quadrant", "sharpe", "volatilidad", "max_drawdown"]
        cols = [c for c in cols if c in df_combined.columns]
        print(df_combined[cols].to_string(index=False))

    def _print_diagnostico_cuadrante(self):
        """Imprime diagnóstico por cuadrante con recomendaciones."""
        df_sel = normalize_columns(self.quality_df.copy())
        df_meta = normalize_columns(self.df_fondos[["Ticker", "Nombre"]].copy())
        df_quad = normalize_columns(self.df[["Ticker", "quadrant"]].copy())

        df_combined = df_sel.merge(df_meta, on="ticker", how="left")
        df_combined = df_combined.merge(df_quad, on="ticker", how="left")
        df_combined = df_combined.sort_values(by="sharpe", ascending=False)

        print("\n" + "="*70)
        print(" DIAGNÓSTICO POR CUADRANTE")
        print("="*70 + "\n")

        def describe_quadrant(q):
            if q == 1:
                return "Cuadrante 1 → Duración corta + Alta calidad. Muy defensivo."
            if q == 2:
                return "Cuadrante 2 → Duración larga + Alta calidad. Beneficia en bajadas de tipos."
            if q == 3:
                return "Cuadrante 3 → Duración corta + Baja calidad. Cupón atractivo, riesgo moderado."
            if q == 4:
                return "Cuadrante 4 → Duración larga + Baja calidad. Muy sensible al ciclo, agresivo."
            return "Cuadrante desconocido."

        for _, row in df_combined.iterrows():
            nombre = row.get('nombre', 'N/D')
            ticker = row.get('ticker')
            q = row.get("quadrant")

            print(f"Fondo: {nombre} ({ticker})")
            print(f"  Cuadrante: {q} — {describe_quadrant(q)}")

            if q == 1:
                print("  Recomendación: Excelente para entornos de incertidumbre o subidas de tipos.")
            elif q == 2:
                print("  Recomendación: Aporta valor si se esperan bajadas de tipos.")
            elif q == 3:
                print("  Recomendación: Bueno para entornos estables con búsqueda de cupón.")
            elif q == 4:
                print("  Recomendación: Solo táctico; riesgo elevado en mercados tensos.")
            else:
                print("  Recomendación: No evaluable.")

            print("-" * 60)

    def _print_recomendacion_global(self):
        """Imprime recomendación global basada en la distribución de cuadrantes."""
        quad = self.summary_by_quadrant()

        print("\n" + "="*70)
        print(" RECOMENDACIÓN GLOBAL DE CARTERA")
        print("="*70 + "\n")

        print("Distribución por cuadrantes:")
        print(quad.to_string())
        print("\n")

        w = {q: quad.loc[q, "weight"] if q in quad.index else 0 for q in [1,2,3,4]}

        print("Interpretación:")

        if w[1] > 0.40:
            print("• Cartera muy defensiva (Cuadrante 1 dominante). Buena protección.")
        if w[2] > 0.30:
            print("• Cartera expuesta a duración larga de alta calidad (Cuadrante 2). Beneficia si bajan tipos.")
        if w[3] > 0.30:
            print("• Cartera orientada a cupón con riesgo moderado (Cuadrante 3).")
        if w[4] > 0.20:
            print("• Atención: exposición elevada al Cuadrante 4 (agresivo).")

        print("\nRecomendación final:")
        if w[1] > max(w[2], w[3], w[4]):
            print("→ Perfil conservador bien construido.")
        elif w[2] > max(w[1], w[3], w[4]):
            print("→ Cartera posicionada para bajadas de tipos.")
        elif w[3] > max(w[1], w[2], w[4]):
            print("→ Cartera buscando cupón con riesgo moderado.")
        else:
            print("→ Cartera agresiva; revisar exposición al Cuadrante 4.")





In [ ]:
# Ruta a tu Excel en Drive
excel_path = "/content/drive/MyDrive/Colab Notebooks/Renta_Fija/cartera_modelo_fondos.xlsx"


# Crear el sistema
pf = FixedIncomePortfolioSystem(excel_path, benchmark="IEAC")  # o el benchmark que quieras

# Cargar datos
pf.load_data()

# Enriquecer fondos
pf.enrich_fondos()

# Procesar
pf.process()

# Analizar calidad
pf.analyze_quality()


In [ ]:
# =============================================================================
# EJEMPLO DE USO SIMPLIFICADO
# =============================================================================

def main():
    """Función principal que ejecuta todo el análisis."""

    # Inicializar sistema
    system = FixedIncomePortfolioSystem("cartera_modelo_fondos.xlsx")

    # Cargar y procesar datos
    system.load_data()
    system.enrich_fondos()
    system.process()

    # Análisis global de la cartera
    system.analyze_portfolio_global()

    # Análisis de calidad
    system.analyze_quality()

    # Contexto de mercado
    market = MarketContext(
        regime="stable_strong",
        as_of_date=datetime.now(),
        comment="Entorno de tipos estables con inflación controlada"
    )

    # Generar informes completos
    system.generate_reports(market)

    # Guardar en Excel
    system.write_to_excel()

    print("\n✅ Análisis completado con éxito")


if __name__ == "__main__":
    main()


✅ Fondos enriquecidos: 6 fondos procesados
Rentabilidad media: 7.25%

 INFORME GLOBAL DE LA CARTERA

Valor inicial: 1.0002
Valor final:   1.1101
Rentabilidad total: 10.99%
CAGR: 4.43%
Volatilidad: 1.03%
Max Drawdown: -0.72%
Sharpe: 3.231
Sortino: 5.144

Diagnóstico cualitativo:
- Rentabilidad total sólida.
- Crecimiento anual compuesto saludable.
- Volatilidad muy baja, perfil conservador.
- Caídas muy controladas.
- Excelente relación rentabilidad/riesgo.
- Buen control de caídas.

 INFORME DE CARTERA DE RENTA FIJA

Distribución por cuadrantes:
            total_value  total_invested    weight  avg_return  accumulated_return
quadrant                                                                         
1         283378.359795   261225.353241  0.369357    0.085644            0.084804
3         483842.601776   454838.638306  0.630643    0.059320            0.063768


Diagnóstico:
- Diagnóstico a fecha: 2026-01-30
- Comentario: Entorno de tipos estables con inflación controlada
- Ent

# Advertencia legal y financiera.
<font color='blue'>

Los contenidos, datos, análisis y herramientas disponibles en este sitio web tienen un propósito **exclusivamente educativo e informativo**. No constituyen asesoramiento financiero, recomendación personalizada de inversión ni oferta de compra o venta de valores.

Aunque se ha procurado ofrecer información clara, útil y actualizada, **no se garantiza la precisión, integridad ni vigencia** El uso de scripts en Python y de las estrategias de gestión presentadas es **responsabilidad exclusiva del usuario.**

El autor de esta web no asume ninguna responsabilidad por decisiones de inversión, pérdidas económicas o daños derivados del uso de la información, herramientas o estrategias aquí expuestas.

📉 **El rendimiento pasado no garantiza resultados futuros.** Toda inversión en mercados financieros conlleva riesgos, y ninguna estrategia —por sofisticada que sea— puede asegurar beneficios. Es esencial aplicar una adecuada gestión del capital y evaluar cuidadosamente cada decisión.

Se recomienda **consultar con profesionales financieros cualificados** antes de tomar decisiones relevantes de inversión o trading.
    </font>